In [3]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from groq import Groq

# Expanding the dataset with more entries
data = [
    {"Symptoms": "Fever, Cough, Fatigue", "Disease": "Flu", "Risk Factors": "Smoking", "Medical History": "Asthma"},
    {"Symptoms": "Chest Pain, Short Breath", "Disease": "Heart Attack", "Risk Factors": "High BP, Diabetes", "Medical History": "Hypertension, High Cholesterol"},
    {"Symptoms": "Joint Pain, Swelling", "Disease": "Arthritis", "Risk Factors": "Obesity", "Medical History": "Family History of Arthritis"},
    {"Symptoms": "Rash, Itching, Swelling", "Disease": "Allergy", "Risk Factors": "Seasonal Allergies", "Medical History": "Hay Fever"},
    {"Symptoms": "Severe Headache, Nausea", "Disease": "Migraine", "Risk Factors": "Stress, Lack of Sleep", "Medical History": "Chronic Migraines"},
    {"Symptoms": "High Fever, Chills, Sweating", "Disease": "Malaria", "Risk Factors": "Mosquito Bites", "Medical History": "Travel to Endemic Areas"},
    {"Symptoms": "Abdominal Pain, Bloating", "Disease": "IBS", "Risk Factors": "Stress, Poor Diet", "Medical History": "Irritable Bowel Syndrome"},
    {"Symptoms": "Frequent Urination, Excess Thirst", "Disease": "Diabetes", "Risk Factors": "Obesity, Family History", "Medical History": "Pre-diabetes"},
    {"Symptoms": "Difficulty Breathing, Wheezing", "Disease": "Asthma", "Risk Factors": "Allergens, Air Pollution", "Medical History": "Chronic Asthma"},
    {"Symptoms": "Memory Loss, Confusion", "Disease": "Alzheimer's", "Risk Factors": "Aging, Genetics", "Medical History": "Family History of Dementia"},
    {"Symptoms": "Unexplained Weight Loss, Night Sweats", "Disease": "Tuberculosis", "Risk Factors": "Weakened Immune System", "Medical History": "Exposure to TB Patients"},
    {"Symptoms": "Painful Urination, Pelvic Pain", "Disease": "UTI", "Risk Factors": "Dehydration, Poor Hygiene", "Medical History": "Frequent UTIs"},
    {"Symptoms": "Nausea, Vomiting, Diarrhea", "Disease": "Food Poisoning", "Risk Factors": "Contaminated Food", "Medical History": "None"},
    {"Symptoms": "Persistent Cough, Weight Loss", "Disease": "Lung Cancer", "Risk Factors": "Smoking, Pollution", "Medical History": "Chronic Bronchitis"},
    {"Symptoms": "Swollen Lymph Nodes, Fatigue", "Disease": "Lymphoma", "Risk Factors": "Genetics, Viral Infections", "Medical History": "None"},
]

# Convert to DataFrame
df = pd.DataFrame(data)

# Initialize a new graph
G = nx.DiGraph()

# Add nodes & edges
for _, row in df.iterrows():
    # Add disease node
    disease = row["Disease"]
    G.add_node(disease, node_type="Disease")
    
    # Process symptoms
    for symptom in row["Symptoms"].split(", "):
        # Add symptom node if it doesn't exist
        G.add_node(symptom, node_type="Symptom")
        # Connect symptom to disease
        G.add_edge(symptom, disease, relation="INDICATES")
    
    # Process risk factors
    for risk in row["Risk Factors"].split(", "):
        G.add_node(risk, node_type="RiskFactor")
        G.add_edge(risk, disease, relation="INCREASES_RISK_OF")
    
    # Process medical history
    for history in row["Medical History"].split(", "):
        if history != "None":  # Skip empty history
            G.add_node(history, node_type="MedicalHistory")
            G.add_edge(history, disease, relation="ASSOCIATED_WITH")

def query_disease(symptoms, graph):
    """Find diseases that might be related to the given symptoms."""
    possible_diseases = set()
    for symptom in symptoms:
        if symptom in graph:
            for neighbor in graph.successors(symptom):
                # Check if the neighbor is a disease
                if graph.nodes[neighbor].get("node_type") == "Disease":
                    possible_diseases.add(neighbor)
    
    return list(possible_diseases)

def extract_symptoms(user_input, graph):
    """Extract symptoms from user input text."""
    # Convert to lowercase for comparison
    input_lower = user_input.lower()
    
    # Get all symptom nodes from the graph
    all_symptoms = [node for node, data in graph.nodes(data=True) 
                   if data.get("node_type") == "Symptom"]
    
    # Find symptoms mentioned in the input
    extracted = []
    for symptom in all_symptoms:
        if symptom.lower() in input_lower:
            extracted.append(symptom)
    
    return extracted

# Example User Query
user_input = "I have Weight Loss."
extracted_symptoms = extract_symptoms(user_input, G)
print(f"Extracted symptoms: {extracted_symptoms}")

diseases = query_disease(extracted_symptoms, G)
print("Possible Diseases:", diseases)

# Optional: Print graph statistics
print(f"\nGraph Statistics:")
print(f"Total nodes: {G.number_of_nodes()}")
print(f"Total edges: {G.number_of_edges()}")




Extracted symptoms: ['Weight Loss']
Possible Diseases: ['Lung Cancer']

Graph Statistics:
Total nodes: 78
Total edges: 72


Using LLM for Query Analyst

In [4]:
import os
import pandas as pd
import networkx as nx
from groq import Groq

# Set up Groq API client (Replace 'your_groq_api_key' with your actual API key)
groq_api_key = "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"
client = Groq(api_key=groq_api_key)

# Expanded dataset for knowledge graph
data = [
    {"Symptoms": "Fever, Cough, Fatigue", "Disease": "Flu", "Risk Factors": "Smoking", "Medical History": "Asthma"},
    {"Symptoms": "Chest Pain, Short Breath", "Disease": "Heart Attack", "Risk Factors": "High BP, Diabetes", "Medical History": "Hypertension, High Cholesterol"},
    {"Symptoms": "Joint Pain, Swelling", "Disease": "Arthritis", "Risk Factors": "Obesity", "Medical History": "Family History of Arthritis"},
    {"Symptoms": "Rash, Itching, Swelling", "Disease": "Allergy", "Risk Factors": "Seasonal Allergies", "Medical History": "Hay Fever"},
    {"Symptoms": "Severe Headache, Nausea", "Disease": "Migraine", "Risk Factors": "Stress, Lack of Sleep", "Medical History": "Chronic Migraines"},
    {"Symptoms": "High Fever, Chills, Sweating", "Disease": "Malaria", "Risk Factors": "Mosquito Bites", "Medical History": "Travel to Endemic Areas"},
    {"Symptoms": "Abdominal Pain, Bloating", "Disease": "IBS", "Risk Factors": "Stress, Poor Diet", "Medical History": "Irritable Bowel Syndrome"},
    {"Symptoms": "Frequent Urination, Excess Thirst", "Disease": "Diabetes", "Risk Factors": "Obesity, Family History", "Medical History": "Pre-diabetes"},
    {"Symptoms": "Difficulty Breathing, Wheezing", "Disease": "Asthma", "Risk Factors": "Allergens, Air Pollution", "Medical History": "Chronic Asthma"},
    {"Symptoms": "Memory Loss, Confusion", "Disease": "Alzheimer's", "Risk Factors": "Aging, Genetics", "Medical History": "Family History of Dementia"},
    {"Symptoms": "Unexplained Weight Loss, Night Sweats", "Disease": "Tuberculosis", "Risk Factors": "Weakened Immune System", "Medical History": "Exposure to TB Patients"},
    {"Symptoms": "Painful Urination, Pelvic Pain", "Disease": "UTI", "Risk Factors": "Dehydration, Poor Hygiene", "Medical History": "Frequent UTIs"},
    {"Symptoms": "Nausea, Vomiting, Diarrhea", "Disease": "Food Poisoning", "Risk Factors": "Contaminated Food", "Medical History": "None"},
    {"Symptoms": "Persistent Cough, Weight Loss", "Disease": "Lung Cancer", "Risk Factors": "Smoking, Pollution", "Medical History": "Chronic Bronchitis"},
    {"Symptoms": "Swollen Lymph Nodes, Fatigue", "Disease": "Lymphoma", "Risk Factors": "Genetics, Viral Infections", "Medical History": "None"},
]

# Convert to DataFrame
df = pd.DataFrame(data)

# Initialize a directed graph
G = nx.DiGraph()

# Populate the knowledge graph
for _, row in df.iterrows():
    disease = row["Disease"]
    G.add_node(disease, node_type="Disease")
    
    for symptom in row["Symptoms"].split(", "):
        symptom_lower = symptom.lower()  # Convert symptoms to lowercase
        G.add_node(symptom_lower, node_type="Symptom")
        G.add_edge(symptom_lower, disease, relation="INDICATES")
    
    for risk in row["Risk Factors"].split(", "):
        G.add_node(risk, node_type="RiskFactor")
        G.add_edge(risk, disease, relation="INCREASES_RISK_OF")
    
    for history in row["Medical History"].split(", "):
        if history != "None":
            G.add_node(history, node_type="MedicalHistory")
            G.add_edge(history, disease, relation="ASSOCIATED_WITH")

# Function to query possible diseases from symptoms
def query_disease(symptoms, graph):
    """Find diseases associated with the given symptoms (case-insensitive)."""
    possible_diseases = set()
    for symptom in symptoms:
        symptom_lower = symptom.lower()  # Normalize input symptom
        if symptom_lower in graph:
            for neighbor in graph.successors(symptom_lower):
                if graph.nodes[neighbor].get("node_type") == "Disease":
                    possible_diseases.add(neighbor)
    return list(possible_diseases)

# Groq LLM-powered symptom extraction
def extract_symptoms_groq(user_input):
    """Extract symptoms using Groq LLM and convert to lowercase."""
    
    prompt = f"""
    You are a medical assistant trained to identify symptoms from patient statements.
    Extract all symptoms mentioned in the following statement:

    Patient: "{user_input}"

    Provide only a comma-separated list of symptoms.
    """
    
    response = client.chat.completions.create(
        model="llama-3.3-70b-specdec",
        messages=[{"role": "system", "content": "You are a medical AI assistant."},
                  {"role": "user", "content": prompt}],
        temperature=0.2
    )
    
    extracted_symptoms = response.choices[0].message.content.strip()
    
    # Convert extracted symptoms to lowercase for consistency
    symptoms_list = [s.strip().lower() for s in extracted_symptoms.split(",") if s.strip()]
    
    return symptoms_list

# Simulated user input
user_input = "I have been experiencing Weight Loss and a persistent cough."

# Extract symptoms using Groq
extracted_symptoms = extract_symptoms_groq(user_input)
print(f"Extracted Symptoms: {extracted_symptoms}")

# Find possible diseases
diseases = query_disease(extracted_symptoms, G)
print(f"Possible Diseases: {diseases}")

# Print graph statistics
print(f"\nGraph Statistics:")
print(f"Total nodes: {G.number_of_nodes()}")
print(f"Total edges: {G.number_of_edges()}")


Extracted Symptoms: ['weight loss', 'persistent cough']
Possible Diseases: ['Lung Cancer']

Graph Statistics:
Total nodes: 78
Total edges: 72


# Minor Upgrade

In [1]:
import os
import pandas as pd
import networkx as nx
from groq import Groq
from dotenv import load_dotenv

# Load .env file
load_dotenv()

# Load Groq API key securely
groq_api_key = os.getenv("GROQ_API_KEY")  # Replace with your API key or set it in the environment
client = Groq(api_key=groq_api_key)

# Expanded dataset for knowledge graph
data = [
    {"Symptoms": "Fever, Cough, Fatigue", "Disease": "Flu", "Risk Factors": "Smoking", "Medical History": "Asthma"},
    {"Symptoms": "Chest Pain, Short Breath", "Disease": "Heart Attack", "Risk Factors": "High BP, Diabetes", "Medical History": "Hypertension, High Cholesterol"},
    {"Symptoms": "Joint Pain, Swelling", "Disease": "Arthritis", "Risk Factors": "Obesity", "Medical History": "Family History of Arthritis"},
    {"Symptoms": "Rash, Itching, Swelling", "Disease": "Allergy", "Risk Factors": "Seasonal Allergies", "Medical History": "Hay Fever"},
    {"Symptoms": "Severe Headache, Nausea", "Disease": "Migraine", "Risk Factors": "Stress, Lack of Sleep", "Medical History": "Chronic Migraines"},
    {"Symptoms": "High Fever, Chills, Sweating", "Disease": "Malaria", "Risk Factors": "Mosquito Bites", "Medical History": "Travel to Endemic Areas"},
    {"Symptoms": "Abdominal Pain, Bloating", "Disease": "IBS", "Risk Factors": "Stress, Poor Diet", "Medical History": "Irritable Bowel Syndrome"},
    {"Symptoms": "Frequent Urination, Excess Thirst", "Disease": "Diabetes", "Risk Factors": "Obesity, Family History", "Medical History": "Pre-diabetes"},
    {"Symptoms": "Difficulty Breathing, Wheezing", "Disease": "Asthma", "Risk Factors": "Allergens, Air Pollution", "Medical History": "Chronic Asthma"},
    {"Symptoms": "Memory Loss, Confusion", "Disease": "Alzheimer's", "Risk Factors": "Aging, Genetics", "Medical History": "Family History of Dementia"},
    {"Symptoms": "Unexplained Weight Loss, Night Sweats", "Disease": "Tuberculosis", "Risk Factors": "Weakened Immune System", "Medical History": "Exposure to TB Patients"},
    {"Symptoms": "Painful Urination, Pelvic Pain", "Disease": "UTI", "Risk Factors": "Dehydration, Poor Hygiene", "Medical History": "Frequent UTIs"},
    {"Symptoms": "Nausea, Vomiting, Diarrhea", "Disease": "Food Poisoning", "Risk Factors": "Contaminated Food", "Medical History": "None"},
    {"Symptoms": "Persistent Cough, Weight Loss", "Disease": "Lung Cancer", "Risk Factors": "Smoking, Pollution", "Medical History": "Chronic Bronchitis"},
    {"Symptoms": "Swollen Lymph Nodes, Fatigue", "Disease": "Lymphoma", "Risk Factors": "Genetics, Viral Infections", "Medical History": "None"},
]

# Convert to DataFrame
df = pd.DataFrame(data)

# Initialize a directed graph
G = nx.DiGraph()

# Populate the knowledge graph
for _, row in df.iterrows():
    disease = row["Disease"]
    G.add_node(disease, node_type="Disease")
    
    for symptom in row["Symptoms"].split(", "):
        symptom_lower = symptom.lower()
        G.add_node(symptom_lower, node_type="Symptom")
        G.add_edge(symptom_lower, disease, relation="INDICATES")
    
    for risk in row["Risk Factors"].split(", "):
        G.add_node(risk, node_type="RiskFactor")
        G.add_edge(risk, disease, relation="INCREASES_RISK_OF")
    
    for history in row["Medical History"].split(", "):
        if history != "None":
            G.add_node(history, node_type="MedicalHistory")
            G.add_edge(history, disease, relation="ASSOCIATED_WITH")

# Function to query possible diseases from symptoms
def query_disease(symptoms, graph):
    """Find diseases associated with the given symptoms (case-insensitive)."""
    possible_diseases = set()
    
    for symptom in symptoms:
        symptom_lower = symptom.lower()
        if symptom_lower in graph:
            possible_diseases.update(
                neighbor for neighbor in graph.successors(symptom_lower)
                if graph.nodes[neighbor].get("node_type") == "Disease"
            )
    
    return list(possible_diseases)

# Groq LLM-powered symptom extraction
def extract_symptoms_groq(user_input):
    """Extract symptoms using Groq LLM with error handling."""
    
    prompt = f"""
    You are a medical assistant trained to identify symptoms from patient statements.
    Extract all symptoms mentioned in the following statement:

    Patient: "{user_input}"

    Provide only a comma-separated list of symptoms.
    """
    
    try:
        response = client.chat.completions.create(
            model="llama-3.3-70b-specdec",
            messages=[
                {"role": "system", "content": "You are a medical AI assistant."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.2
        )

        if response.choices and response.choices[0].message.content:
            extracted_symptoms = response.choices[0].message.content.strip()
            symptoms_list = [s.strip().lower() for s in extracted_symptoms.split(",") if s.strip()]
            return symptoms_list

    except Exception as e:
        print(f"Error during symptom extraction: {e}")
    
    return []

# Simulated user input
user_input = "I have been experiencing High Fever and Cough."

# Extract symptoms using Groq
extracted_symptoms = extract_symptoms_groq(user_input)
print(f"Extracted Symptoms: {extracted_symptoms}")

# Find possible diseases
diseases = query_disease(extracted_symptoms, G)
print(f"Possible Diseases: {diseases}")

# Print graph statistics
print(f"\nGraph Statistics:")
print(f"Total nodes: {G.number_of_nodes()}")
print(f"Total edges: {G.number_of_edges()}")


Extracted Symptoms: ['high fever', 'cough']
Possible Diseases: ['Flu', 'Malaria']

Graph Statistics:
Total nodes: 78
Total edges: 72


# Using Fuzzywuzzy for better string matching

In [1]:
import pandas as pd
import numpy as np
import os
import networkx as nx
from groq import Groq
from dotenv import load_dotenv
import re
from fuzzywuzzy import fuzz, process

# Load environment variables from the .env file
load_dotenv()

# Load Groq API key securely
groq_api_key = os.getenv("GROQ_API_KEY")

# Loading the Groq key
client = Groq(api_key=groq_api_key)

# Sample data for the dataset for Clinical Decision
data = {
    "Symptoms": [
        "Fever, cough, fatigue", "Headache, nausea, blurred vision",
        "Chest pain, shortness of breath, fatigue", "Abdominal pain, nausea, vomiting",
        "Joint pain, swelling, rash", "Cough, wheezing, shortness of breath",
        "Abdominal bloating, diarrhea, weight loss", "Severe headache, vomiting, stiff neck",
        "Fatigue, dizziness, palpitations", "Excessive thirst, frequent urination, fatigue",
        "Weight loss, excessive sweating, tremors", "Shortness of breath, fatigue, swollen legs",
        "Muscle weakness, cramps, headache", "Blurred vision, numbness, tingling in hands and feet",
        "Sudden weight gain, increased appetite, dry skin", "Painful urination, blood in urine, fever",
        "Persistent cough, night sweats, unintentional weight loss", "Severe abdominal pain, loss of appetite, jaundice",
        "Extreme thirst, frequent urination, dry mouth", "Pain in the lower back, difficulty in urinating","Severe Cough, Cold, Running Nose"
    ],
    "Disease": [
        "COVID-19", "Migraine", "Heart Attack", "Gastroenteritis", "Rheumatoid Arthritis",
        "Asthma", "Irritable Bowel Syndrome (IBS)", "Meningitis", "Anemia", "Diabetes", "Hyperthyroidism",
        "Congestive Heart Failure (CHF)", "Electrolyte Imbalance", "Multiple Sclerosis", "Hypothyroidism",
        "Urinary Tract Infection (UTI)", "Tuberculosis", "Liver Disease", "Diabetic Ketoacidosis", "Kidney Infection", "Flu"
    ],
    "Risk Factors": [
        "Recent travel, contact with infected person", "Stress, lack of sleep, dehydration",
        "High blood pressure, smoking, diabetes", "Food poisoning, contaminated water", "Family history, smoking, obesity",
        "Family history, allergens, pollution", "High-fat diet, stress, sedentary lifestyle", "Close contact with infected individuals, immunocompromised",
        "Iron deficiency, chronic blood loss, vitamin B12 deficiency", "Obesity, family history, physical inactivity", "Family history, autoimmune diseases, iodine deficiency",
        "High blood pressure, diabetes, smoking", "Dehydration, excessive alcohol intake, kidney disease", "Family history of autoimmune diseases, fatigue",
        "Iodine deficiency, autoimmune diseases", "Increased age, dehydration, low thyroid hormone levels", "Sexually active, poor hygiene, frequent UTIs",
        "Immune system suppression, history of exposure to TB", "Excessive alcohol intake, viral infections, obesity", "Diabetes, dehydration, infection","Climate Change"
    ],
    "Medical History": [
        "No prior significant medical history", "No prior significant medical history",
        "Previous history of hypertension", "No prior significant medical history", "Family history of autoimmune diseases",
        "History of seasonal allergies", "Previous IBS diagnosis", "No prior significant medical history", "Previous episodes of fatigue and dizziness",
        "No prior significant medical history", "No prior significant medical history", "Previous heart disease diagnosis",
        "History of dehydration and kidney stones", "Family history of neurological disorders", "History of hypothyroidism",
        "Previous UTI, no other major concerns", "No prior significant medical history", "Previous liver problems, high alcohol consumption",
        "History of dehydration and poor blood sugar control", "Previous kidney infections, recurrent UTIs","significant medical history"
    ]
}

# Create a DataFrame from the data
df = pd.DataFrame(data)

# Initialize a directed graph
G = nx.DiGraph()

# Populate the knowledge graph
for _,row in df.iterrows():
    disease = row["Disease"]
    G.add_node(disease, node_type="Disease")

    for symptom in row['Symptoms'].split(","):
        symptom_lower = symptom.strip().lower()
        G.add_node(symptom_lower,node_type="Symptom")
        G.add_edge(symptom_lower,disease,relation="INDICATES")

    for risk in row["Risk Factors"].split(","):
        G.add_node(risk.strip(),node_type="RiskFactor")
        G.add_edge(risk.strip(),disease,relation="Increases_RISK_OF")

    for history in row["Medical History"].split(","):
        if history != "None":
            G.add_node(history.strip(),node_type="MedicalHistory")
            G.add_edge(history.strip(),disease,relation="ASSOCIATED_WITH")


# Function to normalize symptoms using fuzzy matching
def normalize_symptoms(extracted_symptoms, graph):
    """Normalize symptoms to match graph format using fuzzy matching"""
    normalized_symptoms = []
    graph_symptoms = [node for node in graph.nodes() if graph.nodes[node].get("node_type") == "Symptom"]
    
    for symptom in extracted_symptoms:
        # Find the best match in the graph
        best_match = process.extractOne(symptom, graph_symptoms, scorer=fuzz.token_sort_ratio)
        
        if best_match and best_match[1] > 50:  # Adjust the threshold as needed
            normalized_symptoms.append(best_match[0])
        else:
            normalized_symptoms.append(symptom)  # If no good match, keep the original
    
    return normalized_symptoms


# Function to query possible diseases from symptoms using neighbour

def query_disease(symptoms, graph):
    """Find diseases associated with the given symptoms"""
    possible_diseases = set()

    for symptom in symptoms:
        if symptom in graph:
            possible_diseases.update(neighbor for neighbor in graph.successors(symptom)
            if graph.nodes[neighbor].get("node_type") == "Disease")
    return list(possible_diseases) 


# LLM to extract Symptoms from user query

def extract_symptoms_groq(user_input):
    """Extract Symptoms using LLM"""

    prompt = f"""
    You are a medical assistant trained to identify symptoms from patient statements.
    Extract all symptoms mentioned in the following statement:
    
    Patient: "{user_input}"

    Provide only a comma-seperated list of symptoms.

    """
    
    try:
        response = client.chat.completions.create(
            model="llama-3.3-70b-specdec",
            messages=[
                {"role":"system","content":"you are a Medical AI Assistant."},
                {"role":"user","content":prompt}
            ],
            temperature=0.2
        )

        if response.choices and response.choices[0].message.content:
            extracted_symptoms=response.choices[0].message.content.strip()
            extracted_symptoms_list = [s.strip().lower() for s in extracted_symptoms.split(",") if s.strip()]
            
            # Normalize symptoms using fuzzy matching
            normalized_symptoms = normalize_symptoms(extracted_symptoms_list, G)
            return normalized_symptoms

    except Exception as e:
        print(f"Error during symptom extraction:{e}")  

    return []


# Enter your required user query

user_input = "Hello Doctor I have severe pain while urinating"


# Extract symptoms using Groq
extracted_symptoms = extract_symptoms_groq(user_input)
print(f"Extracted Symptoms:{extracted_symptoms}")

# Find possible diseases
diseases = query_disease(extracted_symptoms,G)
print(f"Possible Diseases: {diseases}")

# Print graph statistics
print(f"\nGraph Statistics:")
print(f"Total nodes: {G.number_of_nodes()}")
print(f"Total edges: {G.number_of_edges()}")    


c:\Users\Pritish\anaconda3\envs\ML7\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Extracted Symptoms:['painful urination']
Possible Diseases: ['Urinary Tract Infection (UTI)']

Graph Statistics:
Total nodes: 129
Total edges: 141


#  Using Crew AI Agents for better Decision/Probablity/Priority and Precautions

In [2]:
import os
import pandas as pd
import networkx as nx
from dotenv import load_dotenv
from groq import Groq
from langchain_groq import ChatGroq
from crewai import Agent, Task, Crew

# Load environment variables
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

# Initialize Groq LLM
llm = ChatGroq(model="llama-3.3-70b-specdec", temperature=0.2, groq_api_key=groq_api_key)

### CrewAI Agents ###
diagnosis_agent = Agent(
    role="Medical Diagnostician",
    goal="Analyze symptoms and determine disease probability.",
    backstory="You are a highly skilled medical diagnostician with expertise in analyzing symptoms and calculating disease probabilities.",
    llm=llm,
    memory=True
)

priority_agent = Agent(
    role="Triage Nurse",
    goal="Determine the urgency level based on the disease probability.",
    backstory="You are an experienced triage nurse, skilled at assessing patient urgency based on clinical data.",
    llm=llm,
    memory=True
)

precaution_agent = Agent(
    role="Clinical Advisor",
    goal="Suggest precautions based on disease probability and severity.",
    backstory="You are a clinical advisor who provides medical recommendations and precautions based on diagnosis results.",
    llm=llm,
    memory=True
)

### CrewAI Tasks ###
diagnosis_task = Task(
    description=f"Based on the symptoms {extracted_symptoms}, analyze and determine the probability of each possible disease {diseases}. Provide a probability score for each.",
    agent=diagnosis_agent,
    expected_output="A list of possible diseases with their probability scores."
)

priority_task = Task(
    description="Based on the probabilities of the diseases, determine which condition needs immediate medical attention.",
    agent=priority_agent,
    expected_output="A priority level for the patient's condition (e.g., High, Medium, Low)."
)

precaution_task = Task(
    description="Based on the diagnosed diseases and their probability scores, provide a simple and easy to understand structured answer.",
    agent=precaution_agent,
    expected_output="A structured response, listing key precautions concisely."
)

### Crew Execution ###
medical_crew = Crew(
    agents=[diagnosis_agent, priority_agent, precaution_agent],
    tasks=[diagnosis_task, priority_task, precaution_task],
    #max_iterations=50,  # Increase iteration limit
    step_callback=None,  # Disable step-by-step pause
    timeout=300  # Set timeout in seconds
)

results = medical_crew.kickoff()
print("\nFinal Analysis:")
print(results)



c:\Users\Pritish\anaconda3\envs\ML7\Lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)



Final Analysis:
1. **Prompt Evaluation**: The patient should be evaluated promptly by a healthcare professional to confirm the diagnosis and assess the severity of the condition. This evaluation should include a thorough medical history, physical examination, and laboratory tests such as urinalysis and urine culture.

2. **Antibiotic Therapy**: Given the high probability of UTI, antibiotic therapy should be initiated as soon as possible to prevent further complications. The choice of antibiotic should be based on the patient's medical history, the severity of the infection, and the susceptibility patterns of the likely causative organisms.

3. **Close Monitoring**: Close monitoring of the patient's condition is crucial to quickly identify any signs of complications or worsening of their condition. This includes monitoring of vital signs, urinary symptoms, and laboratory test results.

4. **Hydration**: The patient should be encouraged to drink plenty of fluids, especially water, to he